In [42]:
import csv
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft
from scipy.cluster.vq import vq, kmeans, whiten
from scipy.interpolate import interp1d
from sklearn.cross_validation import KFold
from sklearn.svm import SVC

Xcon = []
Ycon = []
Zcon = []
target = []
with open('dataqm30.csv', 'rb') as csvfile:
    letter = ''
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        letter = row[-1]
        target.append(letter)
        del row[-1]
        frow = np.array(map(float, row))
        Xcon.append(np.array(frow[range(0,len(row),3)]))
        Ycon.append(np.array(frow[range(1,len(row),3)]))
        Zcon.append(np.array(frow[range(2,len(row),3)]))
target = np.array(target)

def inter(Obs, ndots=100):
    interpolated = []
    for obs in Obs:
        x = np.linspace(0, 100, num=obs.shape[0], endpoint=True)
        f = interp1d(x, obs, kind='slinear')
        xnew = np.linspace(0, 100, num=ndots, endpoint=True)
        interpolated.append(f(xnew))
    return interpolated
def interfu(Obs, ndots=100):
    interpolated = []
    for obs in Obs:
        l = len(obs)
        step = 1.0*l/ndots
        xnew = []
        i = 1
        while (i < l-2):
            a = int(i + 0.5)
            xnew.append((obs[a-1]+obs[a]+obs[a+1])/3.0)
            i = i + step
        interpolated.append(xnew)
    return interpolated
def interfuf(Obs, ndots=100):
    interpolated = []
    for obs in Obs:
        l = len(obs)
        step = 1.0*l/ndots
        xnew = []
        i = 0
        while (i < l-1):
            a = int(i + 0.5)
            if (i==0):
                m = (obs[a]+obs[a+1])/2.0
            if (i==l-2):
                m = (obs[a-1]+obs[a])/2.0
            if (i>0) and (i<l-2):
                m = (obs[a-1]+obs[a]+obs[a+1])/3.0
            xnew.append(m)
            i = i + step
        interpolated.append(xnew)
    return interpolated

In [47]:
from sklearn import cross_validation
euc_gram = np.load('euc_kernel.npy')
expgram = np.exp(euc_gram*0.05)
clf = SVC(C=5, kernel='precomputed', decision_function_shape='ovr')
clf.fit(expgram,target)
scores = cross_validation.cross_val_score(clf, expgram, target, cv=10)

print np.mean(scores)


0.970124554245


In [51]:

params = clf.get_params()
sv = clf.support_
nv = clf.n_support_
a  = clf.dual_coef_
b  = clf._intercept_
cs = clf.classes_
start = [sum(nv[:i]) for i in range(len(nv))]
    
end = [start[i] + nv[i] for i in range(len(nv))]
print len(nv)
print start 
print end
cc = []
decision = []
for i in range(len(nv)):
    for j in range(i+1,len(nv)):
        fff = 0
        for p in range(start[j], end[j]):
            fff = fff + a[i][p] * expgram[333][sv[p]]
        for p in range(start[i], end[i]):
            fff = fff + a[j-1][p] * expgram[333][sv[p]]
        cc.append(fff)
        
print len(b)        

decision = [sum(x) for x in zip(cc, b)]
                    
votes = [(i if decision[p] > 0 else j) for p,(i,j) in  
            enumerate((i,j) for i in range(len(cs)) for j in range(i+1,len(cs)))]


print len(a[1])
# Use the functions to predict
#print(predict(params, sv, nv, a, b, cs, ))


26
[0, 30, 57, 85, 116, 145, 173, 201, 226, 252, 280, 309, 337, 366, 392, 421, 448, 485, 517, 545, 574, 604, 630, 660, 689, 718]
[30, 57, 85, 116, 145, 173, 201, 226, 252, 280, 309, 337, 366, 392, 421, 448, 485, 517, 545, 574, 604, 630, 660, 689, 718, 743]
325
743


In [56]:
sten = [cs]
f = open('euc_classes.csv','w')
str1 = ''
for i in range(len(sten)):
    str1 = ''
    for val in sten[i]:
        str1 += str(val)+','
    str1 = str1[:-1]
    str1 += '\n'
    f.write(str1)
f.close()

In [36]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn import cross_validation
def dtw_kernel(X, Y):
    kern = []
    ccc = 0
    for x in X:
        row = []
        for y in Y:
            distance, path = fastdtw(x, y, dist=euclidean)
            #distance,b,c = dtw(x, y)
            #row.append(np.exp(-gamma*distance))
            row.append(-distance)
        kern.append(np.array(row))
        ccc += 1
        if ccc%50==0:
            print ccc
    #print kern
    return np.array(kern)
X = interfu(Xcon,20)
Y = interfu(Ycon,20)
C = []
for i in range(len(X)):
    C.append(np.vstack((X[i],Y[i])).T)
C = np.array(C)
gram = dtw_kernel(C, C)
np.save('dtw_kernel', np.array(gram))

50


In [137]:
np.save('dtw_kernel', np.array(kernel))

In [57]:
re = np.load('euc_kernel.npy')

In [43]:
from numpy.linalg import norm
from scipy.spatial.distance import euclidean
from sklearn import cross_validation
def euc_kernel(X, Y):
    kern = []
    ccc = 0
    for x in X:
        row = []
        for y in Y:
            distance = 0
            for i in range(x.shape[0]):
                distance += euclidean(x[i], y[i])
            #distance,b,c = dtw(x, y)
            #row.append(np.exp(-gamma*distance))
            row.append(-distance)
        kern.append(np.array(row))
        ccc += 1
        if ccc%50==0:
            print ccc
    #print kern
    return np.array(kern)
X = interfuf(Xcon,20)
Y = interfuf(Ycon,20)
C = []
for i in range(len(X)):
    C.append(np.vstack((X[i],Y[i])).T)
C = np.array(C)
euc_gram = euc_kernel(C, C)
np.save('euc_kernel', np.array(euc_gram))

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
